# Arthur Shield Test Harness

The code below can be used to perform a test of the PII, Prompt Injection, Sensitivity, Toxicity, and Hallucination detection capabilities of Arthur Shield.

### Pre-Requisites: 
- You will need to configure which checks you would like to perform within Arthur Shield. See https://shield.docs.arthur.ai/docs/rule-configuration-guide
- You will need to compile a CSV file containing test prompts for each of the rules you would like to evaluate. You can reference the main template CSV in this folder for formatting, and use it to create your own list of prompts. The columns your input CSV should include are as follows (case-sensitive):
    * *Non-hallucination checks*:
        * *id* - A unique ID for the prompt
        * *prompt* - The text you are sending to Shield that you would like to check
        * *flag* - Which flag you expect this prompt to trigger. Must choose one of the following (ENUM): pii, prompt_injection, sensitive_data, toxicity, control
    * *Hallucination checks*:
        * *id* - A unique ID for the prompt
        * *response* - The response from the LLM to be checked - should be mocked/pre-populated for this test
        * *context* - The context which the response is supposed to be referencing
        * *flag* - Which flag you expect this prompt to trigger. Must choose one of the following (ENUM): hallucination, control
- **Notes for creating control prompts/responses**:
  * *For non-hallucination checks*: In the `prompt` column, enter a simple prompt which will not violate any of the rules
  * *For hallucination checks*: Provide context in the `context` column. In the `response` column, provide a response that only contains information that can be found within the context. Example below:
    * `response`: "The main export of Country X is textiles."
    * `context`: "Country X is known for its strong textile industry, which accounts for the majority of its exports."
    * `flag`: control
- Please aim to have at least 15 (and preferably more) prompts for each of the rules you would like to test in order to gather accurate analysis results, including controls

### Output:
This harness will output 2 CSVs: 
- Test output file containing info on the prompt, Shield flags, and latency
- Metrics file containing various evaluation performance metrics for each of the Shield rules


### Configuration
Fill out the config below with the details specific to your Shield instance

In [ ]:
import os
from dotenv import load_dotenv
import shutil
import csv
import json
import pandas as pd
import math
import requests 
from datetime import datetime

from os.path import abspath, join
import sys

utils_path = abspath(join('..', 'utils'))
if utils_path not in sys.path:
    sys.path.append(utils_path)

from shield_utils import setup_env, get_task, create_task_rule, archive_task

load_dotenv()
SHIELD_ENDPOINT = os.getenv('SHIELD_ENDPOINT', 'https://<your-shield-instance>') # Do not include the slash on the end
SHIELD_API_KEY = os.getenv('SHIELD_API_KEY', '<your-shield-api-key>')

# Change to whatever your input file names are
PROMPT_1_PII_INPUT_FILE = 'data/hr_prompt_input_1_pii.csv'
PROMPT_2_PROMPT_INJECTION_INPUT_FILE = 'data/hr_prompt_input_2_prompt_injection.csv'
PROMPT_3_SENSITIVE_DATA_INPUT_FILE = 'data/hr_prompt_input_3_sensitive_data.csv'
PROMPT_4_TOXICITY_INPUT_FILE = 'data/hr_prompt_input_4_toxicity.csv'
PROMPT_5_CONTROL_INPUT_FILE = 'data/hr_prompt_input_5_control.csv'
OUTPUT_FILE = 'output/shield_output_hr.csv'
METRICS_FILE = 'output/shield_test_metrics_hr.csv'

shield_headers = {
    'Authorization': f'Bearer {SHIELD_API_KEY}'
}

setup_env(base_url=SHIELD_ENDPOINT, api_key=SHIELD_API_KEY)

You'll start by setting up a test task. If your Shield instance has default rules, the newly created task will automatically inherit those global rules.

In [ ]:
TASK_ENDPOINT = f"{SHIELD_ENDPOINT}/api/v2/task"
payload = {"name": "test-harness"}

response = requests.post(TASK_ENDPOINT, headers=shield_headers, json=payload).json()
task_id = response["id"]

response

#### Disable default rule(s)

For the purpose of test harness, we will disable the default rules.

In [ ]:
def disable_one_rule(task_id, rule_id):
    endpoint = f"{SHIELD_ENDPOINT}/api/v2/tasks/{task_id}/rules/{rule_id}"
    payload = {"enabled": "false"}
    return requests.patch(endpoint, headers=shield_headers, json=payload)

def enable_one_rule(task_id, rule_id):
    endpoint = f"{SHIELD_ENDPOINT}/api/v2/tasks/{task_id}/rules/{rule_id}"
    payload = {"enabled": "true"}
    return requests.patch(endpoint, headers=shield_headers, json=payload)

def disable_all_rules(task_id):
    get_endpoint = f"{SHIELD_ENDPOINT}/api/v2/tasks/{task_id}"
    response = requests.get(get_endpoint, headers=shield_headers).json()
    rule_ids=[]
    for rule in response["rules"]:
        rule_ids.append(rule["id"])
    for rule_id in rule_ids:
        disable_one_rule(task_id, rule_id)

# Disable all rules from the test-harness task
disable_all_rules(task_id)
get_task(task_id)

#### Add new rules
See https://shield.docs.arthur.ai/docs/rule-configuration-guide for rule setup instructions. Templates are provided below, but you can modify each of these rules as desired.

In [ ]:
pii = {
  "name": "Test PII Rule",
  "type": "PIIDataRule",
  "apply_to_prompt": True,
  "apply_to_response": False,
  "config": {
    "disabled_pii_entities": [
      "CRYPTO",
      "DATE_TIME",
      "CREDIT_CARD",
      "IBAN_CODE",
      "IP_ADDRESS",
      "NRP",
      "LOCATION",
      "PERSON",
      "MEDICAL_LICENSE",
      "US_BANK_NUMBER",
      "US_DRIVER_LICENSE",
      "US_ITIN",
      "US_PASSPORT"
    ]
  }
}
prompt_injection = {
  "name": "Test Prompt Injection Rule",
  "type": "PromptInjectionRule",
  "apply_to_prompt": True,
  "apply_to_response": False
}
toxicity = {
  "name": "Test Toxicity Rule",
  "type": "ToxicityRule",
  "apply_to_prompt": True,
  "apply_to_response": False
}
hallucination = {
  "name": "Test HallucinationV2 Rule",
  "type": "ModelHallucinationRuleV2",
  "apply_to_prompt": False,
  "apply_to_response": True
}
# You can reconfigure the sensitive data rule to your needs by providing positive/negative examples as shown below
sensitive_data = { 
    "name": "Test Sensitive Data Password Rule",
    "type": "ModelSensitiveDataRule",
    "apply_to_prompt": True,
    "apply_to_response": False,
    "config": {
      "examples": [
        {
          "example": "We have prepared ASD research findings at https://needy-guilder.name. For access, use username Ashton_Langosh42 and password 9uZiNfpv9pr8.",
          "result": True
        },
        {
          "example": "Dear Members, the fund allocation data for the next fiscal year will be available on 11/10/1970. The password for the file is 2FDcCtK8ruY3.",
          "result": True
        },
        {
          "example": "Isom_Kling23, once again we require your assistance in revising the zoning laws for the Gwent. We have left the necessary documents and instructions on the FTP server. IP address: 97.167.207.127, login: Timmy_Wintheiser, password: RMvJ4UnmtMqk.",
          "result": True
        },
        {
          "example": "Hello Mrs. Stiedemann, our School Management App's credentials for Wilford.Schmitt are - Username: Theo.Abernathy58, Password: vDConKUVkCF3. Your association through this app will boost our endeavour for your ward's development.",
          "result": True
        },
        {
          "example": "Hi Miss Rippin, kindly confirm the transaction of £1,482.89 for your telemedicine consultation from your account number: 03433297.",
          "result": False
        },
        {
          "example": "We are planning a server maintenance on 22th February at 0 and need to validate the connection to 197.54.143.140. Can you please assist with this?",
          "result": False
        },
        {
          "example": "Hello, This is your Family therapist. It is to notify that a payment is due for the last therapy session. This is payable via the link https://only-contract.com. Please use the credit card number 7307180895158036. The session cost is Дин.100.",
          "result": False
        },
        {
          "example": "Dear Dortha, due to a breach in our system, we advice you to immediately change your password (jkaWzJMMM9gK) for the online school portal.",
          "result": True
        },
        {
          "example": "Professor Marlowe, could you provide us with a lecture concerning the constitutional limitations in the realm of data privacy? In particular, touch upon how constitutional laws protect user data like passwords (e.g., Dte2bg0vfDXX) and credit card information (e.g., 2347580283766376, 209). We believe that your expertise would immensely benefit our students.",
          "result": True
        },
        {
          "example": "Dear Ms.Sigmund.Wehner67, all employees will need to change their passwords. Please make this update by 6 PM today. Spread the word in your Group. Contact us via 186.165.167.26 for any issues.",
          "result": False
        },
        {
          "example": "The name tag for the new staff member, Angel has arrived. You will find it in the Apt. 634.",
          "result": False
        },
        {
          "example": "Hello Daphney, here are the coordinates [-49.3619,150.137] of your host family's place. Also, your secondary contact address will be Suite 747. Please keep this info safe.",
          "result": False
        }
      ]
    }
}

Once you've configured the rule settings, add them to the task:

In [ ]:
pii_rule = create_task_rule(task_id, pii)
pii_rule_id = pii_rule.get('id')

prompt_injection_rule = create_task_rule(task_id, prompt_injection)
prompt_injection_rule_id = prompt_injection_rule.get('id')

toxicity_rule = create_task_rule(task_id, toxicity)
toxicity_rule_id = toxicity_rule.get('id')

hallucination_rule = create_task_rule(task_id, hallucination)
hallucination_rule_id = hallucination_rule.get('id')

sensitive_data_rule = create_task_rule(task_id, sensitive_data)
sensitive_data_rule_id = sensitive_data_rule.get('id')

get_task(task_id)

Rules in Shield are immutable to maintain accurate validation result records. However, they can be deleted and recreated if modification is required..

In [ ]:
# Delete one rule from the test-harness task
def delete_one_rule(task_id, rule_id):
    endpoint = f"{SHIELD_ENDPOINT}/api/v2/tasks/{task_id}/rules/{rule_id}"
    return requests.delete(endpoint, headers=shield_headers)

# Delete all rules from the test-harness task
def delete_all_rules(task_id):
    get_endpoint = f"{SHIELD_ENDPOINT}/api/v2/tasks/{task_id}"
    response = requests.get(get_endpoint, headers=shield_headers).json()
    rule_ids=[]
    for rule in response["rules"]:
        if(rule["scope"] == "task"):
            rule_ids.append(rule["id"])
    for rule_id in rule_ids:
        delete_one_rule(task_id, rule_id)

# delete_one_rule(task_id, <rule_id>)
# delete_all_rules(task_id)

## i. Non-Hallucination Checks

### Support Functions

Shield's response schema can be found on the API docs: https://\<your-shield-instance>/docs#/Default%20Validation/default_validate_prompt_api_v2_validate_prompt_post

In [ ]:
SHIELD_VAL_ENDPOINT = f"{SHIELD_ENDPOINT}/api/v2/tasks/{task_id}/validate_prompt"

def get_firewall_flags(resp_dict):
    # Determines which rules (PII, Prompt Injection, Toxicity, Sensitivity) were flagged
    flags = []
    for rule in resp_dict["rule_results"]:
        if rule["result"] == "Fail":
            if rule["rule_type"] in ["ModelHallucinationRuleV2", "ModelHallucionationRuleV3"] and "hallucination" not in flags:
                # Only used for responses - does not check for other flags
                flags.append("hallucination")
                break
            elif rule["rule_type"] == "PIIDataRule" and "pii" not in flags:
                flags.append("pii")
            elif rule["rule_type"] == "PromptInjectionRule" and "prompt_injection" not in flags:
                flags.append("prompt_injection")
            elif rule["rule_type"] == "ToxicityRule" and "toxicity" not in flags:
                flags.append("toxicity")
            elif rule["rule_type"] == "ModelSensitiveDataRule" and "sensitive_data" not in flags:
                flags.append("sensitive_data")
    return flags

def get_gt_output_match(row):
    # Returns True if GT label matches Shield flags and False otherwise
    if row["ground_truth"] == "control": 
        return False if row["shield_response"] else True
    else:
        return True if row["ground_truth"] in row["shield_response"] else False
    

def get_pii_triggers(shield_response):
    # Isolate the text which triggered the PII rule to fire
    output = []
    for rule in shield_response["rule_results"]:
        if rule["rule_type"]== "PIIDataRule" and rule["result"]=="Fail":
            for flag in rule["details"]["pii_entities"]:
                output.append({
                    "string_trigger": flag["span"],
                    "flag_type": flag["entity"]
                })
    return output

def get_shield_response(row):
    # Calls Shield API and updates the row dictionary based upon the results from Shield
    shield_start = datetime.now()
    response = requests.post(SHIELD_VAL_ENDPOINT, headers=shield_headers, json={"prompt": row["prompt"]})
    shield_end = datetime.now()
    row["latency_ms"] = int((shield_end - shield_start).microseconds/1000)
    row["shield_response"] = get_firewall_flags(response.json())
    row["gt_output_match"] = get_gt_output_match(row)
    row["sub_flags"] = get_pii_triggers(response.json()) if "pii" in row["shield_response"] else None
    return row

# Generate and append Shield response to the output file
def append_shield_response(filename):
    with open(filename, newline='') as infile:
        reader= csv.DictReader(infile)
        file_exists = os.path.isfile(OUTPUT_FILE)
        with open(OUTPUT_FILE, 'a', newline='') as outfile:
            # List of columns for the output file
            fieldnames = [
                "id",
                "prompt",
                "ground_truth",
                "shield_response",
                "gt_output_match",
                "sub_flags",
                "latency_ms"
            ]
            writer= csv.DictWriter(outfile, fieldnames=fieldnames)
            if not file_exists:
                writer.writeheader()
            
            for row in reader:
                out_row = {
                    "id": row["id"],
                    "ground_truth": row["flag"],
                    "prompt": row["prompt"]
                    }
    
                out_row = get_shield_response(out_row)
                writer.writerow(out_row)

### Generate Output File

The output file generated will contain: 
  * *id* - The ID of the corresponding input file prompt
  * *prompt* - The prompt that was passed
  * *ground_truth* - The flag that was expected
  * *shield_response* - The flag that Shield raised
  * *test_passed* - Whether the ground truth label matches the flag that Shield raised (True or False)
  * *sub-flags* - (For PII) Which sentences were flagged and which PII flag was raised
  * *latency_ms* - Latency of Shield call in ms

In [ ]:
# Remove previous output files
output_folder_name = "output"
if os.path.exists(output_folder_name):
    shutil.rmtree(output_folder_name)
os.makedirs(output_folder_name)

# Generate control input validation
append_shield_response(PROMPT_5_CONTROL_INPUT_FILE)
# Disable all rules to analyze one rule at a time
disable_all_rules(task_id)

# Generate PII validations
enable_one_rule(task_id, pii_rule_id)
append_shield_response(PROMPT_1_PII_INPUT_FILE)
disable_one_rule(task_id, pii_rule_id)

# Generate Promopt Injection validations
enable_one_rule(task_id, prompt_injection_rule_id)
append_shield_response(PROMPT_2_PROMPT_INJECTION_INPUT_FILE)
disable_one_rule(task_id, prompt_injection_rule_id)

# Generate sensitive data validations
enable_one_rule(task_id, sensitive_data_rule_id)
append_shield_response(PROMPT_3_SENSITIVE_DATA_INPUT_FILE)
disable_one_rule(task_id, sensitive_data_rule_id)

# Generate toxicity validations
enable_one_rule(task_id, toxicity_rule_id)
append_shield_response(PROMPT_4_TOXICITY_INPUT_FILE)
disable_one_rule(task_id, toxicity_rule_id)

In [ ]:
output_df = pd.read_csv(OUTPUT_FILE)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)
output_df

## ii. Hallucination Checks

The hallucination checks are for validating the LLM generated responses only and require a slightly different input format, so we run those checks separately here.

In [ ]:
HALLUCINATION_INPUT_FILE = 'data/hr_response_input_hallucination_and_control.csv' # Change this to whatever your input file name is 
HALLUCINATION_OUTPUT_FILE = 'output/shield_output_hallucination_hr.csv'

SHIELD_RESP_ENDPOINT = f"{SHIELD_ENDPOINT}/api/v2/tasks/{task_id}/validate_response/"

### Support Functions

In [ ]:
def get_claims(response):
    # Extracts the claims from the Shield flag 
    for rule in response["rule_results"]:
        if rule["rule_type"] not in ["ModelHallucinationRuleV2", "ModelHallucinationRuleV3"]:
            continue
        else:
            if rule["result"] == "Pass":
                return []
            else:
                return rule["details"]["claims"]

def get_inference_id(prompt):
    # Conduct an initial Shield call to get an inference ID for the response endpoint
    response = (requests.post(
        SHIELD_VAL_ENDPOINT, 
        headers=shield_headers, 
        json={"prompt": prompt})).json()
    return str(response["inference_id"])
        
def get_hallucination_shield_response(row):
    # Calls Shield API and updates the row dictionary based upon the results from Shield
    resp_endpoint = SHIELD_RESP_ENDPOINT + get_inference_id("null")
    shield_start = datetime.now()
    response = requests.post(
        resp_endpoint, 
        headers=shield_headers, 
        json={"response": row["llm_response"], "context": row["context"]}
    )
    shield_end = datetime.now()
    row["latency_ms"] = int((shield_end - shield_start).microseconds/1000)
    row["shield_response"] = get_firewall_flags(response.json())
    row["claims"] = get_claims(response.json())
    return row
    

### Generate Output File

In [ ]:
enable_one_rule(task_id, hallucination_rule_id)
with open(HALLUCINATION_INPUT_FILE, newline='') as infile:
    reader= csv.DictReader(infile)
    with open(HALLUCINATION_OUTPUT_FILE, 'w', newline='') as outfile:
        # List of columns for the output file
        fieldnames = [
            "id",
            "context",
            "llm_response",
            "ground_truth",
            "shield_response",
            "claims",
            "latency_ms"
        ]
        writer= csv.DictWriter(outfile, fieldnames=fieldnames)
        writer.writeheader()
        
        for row in reader:
            out_row = {
                "id": row["id"],
                "llm_response": row["response"],
                "context": row["context"],
                "ground_truth": row["flag"]
                }

            out_row = get_hallucination_shield_response(out_row)
            writer.writerow(out_row)
disable_one_rule(task_id, hallucination_rule_id)

In [ ]:
output_df = pd.read_csv(HALLUCINATION_OUTPUT_FILE)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)
output_df

## iii. Analysis

The metrics file generated will contain:
  * True Positive Count
  * False Positive Count
  * Precision
  * Recall
  * Specificity
  * Miss Rate
  * False Positive Rate
  * F1 Score

In [ ]:
def round_row(data_row, precision=3):
    return [round(item, precision) if isinstance(item, (float, int)) else item for item in data_row]

def create_metrics_dict(metric_name):
    if metric_name == "hallucination":
        hdf= pd.read_csv(HALLUCINATION_OUTPUT_FILE)
        metrics = {
            "tp": hdf[
            (hdf["ground_truth"] == "hallucination") & (hdf['shield_response'].apply(lambda x: metric_name in x))
            ].shape[0],
            "fp": hdf[
            (hdf["ground_truth"] == "control") & (hdf['shield_response'].apply(lambda x: metric_name in x))
            ].shape[0],
            "fn": hdf[
            (hdf["ground_truth"] == "hallucination") & (hdf['shield_response'].apply(lambda x: metric_name not in x))
            ].shape[0],
            "tn": hdf[
            (hdf["ground_truth"] == "control") & (hdf['shield_response'].apply(lambda x: metric_name not in x))
            ].shape[0]
        }
    else:
        df = pd.read_csv(OUTPUT_FILE)
        metrics = {
            "tp": df[(df['gt_output_match']==True) & (df['ground_truth']==metric_name)].shape[0],
            "fp": df[(df['ground_truth']!=metric_name) & (df['shield_response'].apply(lambda x: metric_name in x))].shape[0],
            "fn": df[(df['ground_truth']==metric_name) & (df['gt_output_match']==False)].shape[0],
            "tn": df[(df['ground_truth']!=metric_name) & (df['shield_response'].apply(lambda x: metric_name not in x))].shape[0]
        }
    try:
        metrics["prec"] = metrics["tp"]/(metrics["tp"]+metrics["fp"])
    except: 
        metrics["prec"] = "N/A"
    try:
        metrics["recall"] = metrics["tp"]/(metrics["tp"]+metrics["fn"])
    except:
        metrics["recall"] = "N/A"
    return metrics

def run_analysis(checks=["pii", "prompt_injection", "toxicity", "sensitive_data", "hallucination"]):
    # Runs analysis on the selected checks. Runs on all by default
    with open(METRICS_FILE, 'w', newline='') as outfile:
        writer = csv.writer(outfile)
        metrics = []
        for check in checks:
            metrics.append(create_metrics_dict(check))
        writer.writerow([""]+checks)
    
        # True Positive Count
        tpc= ["True Positive Count"]
        for metric in metrics:
            tpc.append(metric["tp"])
        writer.writerow(round_row(tpc))
    
        # False Positive Count
        fpc= ["False Positive Count"]
        for metric in metrics:
            fpc.append(metric["fp"])
        writer.writerow(round_row(fpc))
                        
        # Precision
        prec = ["Precision"]
        for metric in metrics:
            prec.append(metric["prec"])
        writer.writerow(round_row(prec))
        
        # Recall
        recall = ["Recall"]
        for metric in metrics:
            recall.append(metric["recall"])
        writer.writerow(round_row(recall))

        # Specificity
        spec = ["Specificity"]
        for metric in metrics:
            try:
                spec.append(metric["tn"]/(metric["tn"]+metric["fp"]))
            except:
                spec.append("N/A")
        writer.writerow(round_row(spec))
    
        # Miss Rate
        miss = ["Miss Rate"]
        for metric in metrics:
            try:
                miss.append(metric["fn"]/(metric["fn"]+metric["tp"]))
            except:
                miss.append("N/A")
        writer.writerow(round_row(miss))
        
        # False Positive Rate
        fpr = ["False Positive Rate"]
        for metric in metrics:
            try:
                fpr.append(metric["fp"]/(metric["fp"]+metric["tn"]))
            except:
                fpr.append("N/A")
        writer.writerow(round_row(fpr))
    
        # F1 Score
        f1 = ["F1 Score"]
        for metric in metrics:
            try:
                f1.append((2*metric["prec"]*metric["recall"])/(metric["prec"]+metric["recall"]))
            except:
                f1.append("N/A")
        writer.writerow(round_row(f1))


The cell below will run analysis on all of the flags included in the list (the list can contain "pii", "prompt_injection", "toxicity", and/or "sensitivity"). **If any of these flags are not enabled in the Shield instance you are evaluating, omit it from the input list to avoid generating errors.**

In [ ]:
run_analysis([
    "pii",
    "prompt_injection",
    "toxicity",
    "sensitive_data",
    "hallucination"
])

## iv. Cleanup

Once you've finished with your testing, you can use the following cell to delete the test task:

In [ ]:
archive_task(task_id)